### Imports

First we need to correctly set the python environment. This is done by adding the top directory of the repository to the Python path. Once that's done, we can import various packages from inside the repository.

In [1]:
import sys, models, yaml, argparse, logging, math, numpy as np, torch, torchvision, sherpa, logging
sys.path.append('/scratch') # This line is equivalent to doing source scripts/source_me.sh in a bash terminal
from torch.utils.data import DataLoader
import MinkowskiEngine as ME
from SparseBase.trainers import SparseTrainer
from glob import glob
from SparseBase import utils
from SparseNOvA import datasets

### Configuring

Most of the training options are set in a configuration YAML file. We're going to load this config, and then the options inside will be passed to the relevent piece of the training framework.

In [2]:
with open('/scratch/SparseNOvA/config/sparse_nova_mobilenet1.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/SparseNOvA/config/sparse_nova_mobilenet1.yaml'

### Loading inputs

Here we load the dataset and the trainer, which is responsible for building the model and overseeing training. There's a block of code which is responsible for slicing the full dataset up into a training dataset and a validation dataset where jitter is applied to training dataset only.

In [3]:
all_nus = sorted(glob('/data/mp5/preselection_v2/nu/*.pt'))
all_cosmics = sorted(glob('/data/mp5/preselection_v2/cosmic/*.pt'))

if len(all_cosmics) > int(0.1 * len(all_nus)):
    all_cosmics = all_cosmics[0:int(0.1*len(all_nus))]

fulllen_nu = len(all_nus)
fulllen_cosmic = len(all_cosmics)

tv_num_nu = math.ceil(fulllen_nu*config['data']['t_v_split'])
tv_num_cosmic = math.ceil(fulllen_cosmic*config['data']['t_v_split'])

splits_nu = np.cumsum([fulllen_nu - tv_num_nu, 0, tv_num_nu])
splits_cos = np.cumsum([fulllen_cosmic - tv_num_cosmic, 0, tv_num_cosmic])
# print(splits_nu)
# print(splits_cos)

train_files = sorted(all_nus[0:splits_nu[1]] + all_cosmics[0:splits_cos[1]])
train_dataset = datasets.get_dataset(filelist=train_files, apply_jitter=True, **config['data'])

valid_files = sorted(all_nus[splits_nu[1]:splits_nu[2]] + all_cosmics[splits_cos[1]:splits_cos[2]])
valid_dataset = datasets.get_dataset(filelist=valid_files, apply_jitter=False, **config['data'])

parameters = [sherpa.Continuous('learning_rate', [1e-5, 1e-1]), sherpa.Continuous('weight_decay', [0.01, 0.1]), sherpa.Discrete('unet_depth', [2, 6])]

trainer = SparseTrainer(**config['trainer'])

alg = sherpa.algorithms.GPyOpt(max_num_trials=50)

study = sherpa.Study(parameters=parameters, algorithm=alg, lower_is_better=True, dashboard_port=8000)

collate = utils.collate_sparse_minkowski_2stack

train_loader = DataLoader(train_dataset, collate_fn=collate, **config['data_loader'], shuffle=True)
valid_loader = DataLoader(valid_dataset, collate_fn=collate, **config['data_loader'], shuffle=False)

### Building the model

The trainer will load the network architecture and compile it into a model.
The second line will keep a log of the trainer for reference. 
The third line will use specific parameters which were successful in a previous training. 

In [4]:
trainer.build_model(**config['model'])
trainer.logger.addHandler(logging.FileHandler('/scratch/SparseNOvA/logs/train.log'))
# trainer.load_state_dict(state_dict='params/checkpoints/model_checkpoint_Minkowski2StackClass_213908357_2e641e404d_haejunoh_005.pth.tar')

### Training!

Once all the setup is done, all that's left is to run training and save some summary statistics to file.

In [ ]:
train_summary = trainer.train(train_loader, valid_data_loader=valid_loader, sherpa_study=study, sherpa_trial=trial, **config['trainer'])
print(train_summary)
torch.save(train_summary, 'summary_test.pt')

loss = 1.36867:  73%|███████▎  | 35119/47954 [3:09:25<1:35:58,  2.23it/s]